<a href="https://colab.research.google.com/github/KAZIRASEL2024/fever-prediction-app/blob/main/Copy_of_DMML_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving framingham.csv to framingham.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


In [3]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/drkhaledmohsin/national-heart-foundation-2023-ecg-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [ ]:
df=pd.read_csv("/content/framingham.csv")
df.head()

In [ ]:
df.shape

In [ ]:
# Count occurrences of 0 and 1
counts = df['TenYearCHD'].value_counts()
print(counts)

In [ ]:
df.isnull().sum()/100

In [ ]:
(df.isnull().sum()) / len(df)*100

In [ ]:
df.info()

In [ ]:
# prompt: detect outliner and handle the outliner from the data set

# Assuming you have a DataFrame named 'df' with your dataset

# Detect outliers using IQR (Interquartile Range) method for numerical features
def detect_outliers_iqr(data):
  outliers = {}
  for col in data.select_dtypes(include=np.number):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers[col] = data[(data[col] < lower_bound) | (data[col] > upper_bound)].index.tolist()
  return outliers

# Handle outliers (replace with median or mean, or remove)
def handle_outliers(data, outliers, method='median'):
  for col, outlier_indices in outliers.items():
    if method == 'median':
      median_val = data[col].median()
      data.loc[outlier_indices, col] = median_val
    elif method == 'mean':
      mean_val = data[col].mean()
      data.loc[outlier_indices, col] = mean_val
    elif method == 'remove':
      data.drop(outlier_indices, inplace=True)
  return data


# Example usage:
outliers_dict = detect_outliers_iqr(df)  # Detect outliers in your DataFrame 'df'
df_no_outliers = handle_outliers(df.copy(), outliers_dict, method='median')  # Replace outliers with median
# You can choose 'mean' or 'remove' as the method for handling outliers

In [ ]:
from google.colab import files
files.download('fever_model.pkl')


In [ ]:
# prompt: drop all the null value from the data data set in column "glucose"

# Drop rows with null values in the 'glucose' column
df.dropna(subset=['glucose'], inplace=True)


In [ ]:
df.isnull().sum()

In [ ]:
# prompt: fill missing numerical value with median and categorical value with mode

for column in df.columns:
  if df[column].dtype == 'object':
    df[column].fillna(df[column].mode()[0], inplace=True)
  elif pd.api.types.is_numeric_dtype(df[column]):
    df[column].fillna(df[column].mean(), inplace=True)


In [ ]:
df.isnull().sum()


In [ ]:
# prompt: def encode_categorical_features(df):
#     label_encoders = {}
#     for col in df.select_dtypes(include=['object']).columns:
#         le = LabelEncoder()
#         df[col] = le.fit_transform(df[col])
#         label_encoders[col] = le
#     return df, label_encoders
# # Encode categorical variables
# df, label_encoders = encode_categorical_features(df)
# is correct for this data set if not apply correct code

def encode_categorical_features(df):
    label_encoders = {}
    for col in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    return df, label_encoders

# Assuming 'df' is your DataFrame
# Encode categorical variables
df, label_encoders = encode_categorical_features(df)


In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
features_to_scale = ['age','education', 'cigsPerDay', 'BPMeds', 'totChol', 'sysBP', 'diaBP', 'BMI','heartRate','glucose']  # Replace with your desired feature names

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit and transform the selected features
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

df.head()

In [ ]:
df.info()

In [ ]:
# prompt: please apply oversampling method for to columns called TenYearCHD

from imblearn.over_sampling import SMOTE

# Separate features and target variable
X = df.drop('TenYearCHD', axis=1)
y = df['TenYearCHD']

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create a new DataFrame with the resampled data
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['TenYearCHD'] = y_resampled

# Print the new class distribution
print(df_resampled['TenYearCHD'].value_counts())

In [ ]:
df.info()

In [ ]:
# Split data
X = df_resampled.drop("TenYearCHD", axis=1)
y = df_resampled["TenYearCHD"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import shap

# Train XGBoost model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Explain predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test)

In [ ]:
# prompt: apply a new model called random forest and take all nessesary step have to be taken

from sklearn.ensemble import RandomForestClassifier

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust n_estimators and other hyperparameters
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

# Print classification report
print(classification_report(y_test, y_pred_rf))

# You can also use SHAP to explain predictions for the Random Forest model
# explainer_rf = shap.Explainer(rf_model)
# shap_values_rf = explainer_rf(X_test)
# shap.summary_plot(shap_values_rf, X_test)


In [ ]:
# prompt: apply a new model called LightGBM

# Install LightGBM if not already installed
!pip install lightgbm

import lightgbm as lgb
from sklearn.metrics import accuracy_score


# Train LightGBM model
lgb_model = lgb.LGBMClassifier()  # You can adjust hyperparameters as needed
lgb_model.fit(X_train, y_train)

# Predict
y_pred_lgb = lgb_model.predict(X_test)
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgb))

# Print classification report
print(classification_report(y_test, y_pred_lgb))

# You can also use SHAP to explain predictions for the LightGBM model (similar to Random Forest)
# explainer_lgb = shap.Explainer(lgb_model)
# shap_values_lgb = explainer_lgb(X_test)
# shap.summary_plot(shap_values_lgb, X_test)

In [ ]:
# prompt: apply  Voting Classifier for those three model

from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier with the three models
voting_clf = VotingClassifier(estimators=[
    ('xgb', model),  # Your XGBoost model
    ('rf', rf_model),  # Your Random Forest model
    ('lgb', lgb_model)  # Your LightGBM model
], voting='soft')  # You can use 'hard' voting as well

# Train the Voting Classifier
voting_clf.fit(X_train, y_train)

# Predict using the Voting Classifier
y_pred_voting = voting_clf.predict(X_test)

# Evaluate the Voting Classifier
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print(classification_report(y_test, y_pred_voting))


In [ ]:
# prompt: make confusion matrix and the roc curve

from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming y_test and y_pred_voting are defined from the previous code

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_voting)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# ROC Curve
y_prob = voting_clf.predict_proba(X_test)[:, 1]  # Probability of the positive class
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the paths to the data directories
data_dir = '/content/national-heart-foundation-2023-ecg-dataset/ECG Data'
categories = ['Abnormal Heartbeat Patients', 'Myocardial Infarction Patients',
              'Normal Person', 'Patient that have History of Myocardial Infraction']

In [ ]:
img_size = 128  # Desired image size for resizing

def load_and_preprocess_images(data_dir, categories, img_size):
    data = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_label = categories.index(category)  # Assign class labels based on category index
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            try:
                img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img_array is not None:
                    resized_img = cv2.resize(img_array, (img_size, img_size))
                    data.append([resized_img, class_label])
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                continue
    return data

# Load and preprocess images
data = load_and_preprocess_images(data_dir, categories, img_size)

# Shuffle the data to ensure random distribution
np.random.shuffle(data)

# Split data into features and labels
X, y = zip(*data)
X = np.array(X).reshape(-1, img_size, img_size, 1)  # Reshape for CNN input
y = np.array(y)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (2318, 128, 128, 1)
Testing data shape: (580, 128, 128, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')  # Output layer with units equal to number of categories
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper right')

plt.show()

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")